#### 환율 변동값 전처리
* 전일 변동율 = (전날종가-전날시가)/전날시가 yesterday_change <br>
* 야간 변동율 = (당일시가-전날종가)/전날종가 night_change <br>
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv("../dataset/usd_krw.csv")

In [4]:
# 고가, 저가 드롭
usd = df.drop(['high','low'], axis=1)

In [5]:
usd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          61 non-null     object
 1   close         61 non-null     object
 2   open          61 non-null     object
 3   close_change  61 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


In [6]:
# 특수문자 제거, 데이터 타입 변경
usd['close'] = usd['close'].str.replace(',','')
usd['open'] = usd['open'].str.replace(',','')


usd[['close','open']] = usd[['close','open']].astype(float)
df['date'] = pd.to_datetime(df['date'])

In [8]:
# 날짜순으로 오름차순 정렬
usd = usd.sort_values(by='date')

In [9]:
# 인덱스 변경
usd.reset_index(drop=True, inplace=True)

In [10]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
usd['usd_yesterday_change'] = round((usd['close']-usd['open'])/usd['open']*100, 2)
usd['usd_yesterday_change'] = usd['usd_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
usd['close_shift'] = usd['close'].shift(1)
usd['usd_night_change'] = round((usd['open']-usd['close_shift'])/usd['close_shift']*100,2)

usd.drop('close_shift',axis=1,inplace=True)

In [12]:
usd

,date,close,open,close_change,usd_yesterday_change,usd_night_change
0,2023-07-28,1272.83,1284.17,-0.85%,NaN,NaN
1,2023-07-31,1277.86,1273.33,0.40%,-0.88,0.04
2,2023-08-01,1289.00,1278.28,0.87%,0.36,0.03
3,2023-08-02,1298.93,1289.40,0.77%,0.84,0.03
4,2023-08-03,1299.01,1299.47,0.01%,0.74,0.04
...,...,...,...,...,...,...
56,2023-10-16,1347.91,1353.08,-0.32%,0.21,0.06
57,2023-10-17,1352.98,1348.25,0.38%,-0.38,0.03
58,2023-10-18,1353.86,1353.55,0.07%,0.35,0.04
59,2023-10-19,1356.02,1355.48,0.16%,0.02,0.12


In [13]:
# 종가 기준 변동율 전처리 및 데이터 형태 변환( 단위 : % )
usd['close_change'] = usd['close_change'].str.replace('%','')
usd['close_change'] = usd['close_change'].astype(float)

In [14]:
usd = usd.loc[1:,]
usd = usd[['date','usd_yesterday_change','usd_night_change']]

In [15]:
usd

,date,usd_yesterday_change,usd_night_change
1,2023-07-31,-0.88,0.04
2,2023-08-01,0.36,0.03
3,2023-08-02,0.84,0.03
4,2023-08-03,0.74,0.04
5,2023-08-04,-0.04,0.07
6,2023-08-07,0.32,0.02
7,2023-08-08,0.23,0.05
8,2023-08-09,0.67,0.02
9,2023-08-10,-0.30,0.00
10,2023-08-11,0.29,0.09


In [30]:
# csv 파일로 새로 저장
os.chdir("C:/Users/LGPC/Desktop/oncoding/Project1/dataset")
usd.to_csv("C:/Users/LGPC/Desktop/oncoding/Project1/dataset/usd_krw_final.csv")